In [1]:
import torch
import monai

from monai.transforms import (
    Compose,
    NormalizeIntensityd,
    RandSpatialCropd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    Activations, AsDiscrete,Resized
)
from torch.utils.tensorboard import SummaryWriter

from dataset import HNTSDataset
from trainer import MedSegTrainer
from monai.metrics import DiceMetric

In [2]:
class Config:
    device = "cuda"
    batch_size = 4
    epochs = 400
    lr_init = 1e-4
    lr_min = 1e-10
    cpu_cores = 8
    weight_decay = 1e-5
    smooth_nr = 0
    smooth_dr = 1e-5


config = Config()

In [3]:
train_transforms = Compose(
    [
        # Normalization and cropping
        RandSpatialCropd(
            keys=["image", "mask"], roi_size=[224, 224, 96], random_size=False
        ),
        RandFlipd(keys=["image", "mask"], prob=0.5, spatial_axis=0),
        RandFlipd(keys=["image", "mask"], prob=0.5, spatial_axis=1),
        RandFlipd(keys=["image", "mask"], prob=0.5, spatial_axis=2),
        NormalizeIntensityd(keys="image", nonzero=True),
        RandScaleIntensityd(keys="image", factors=0.1, prob=1.0),
        RandShiftIntensityd(keys="image", offsets=0.1, prob=1.0),
    ]
)
train_dataset = HNTSDataset("data/train", transform=train_transforms)
train_loader = monai.data.DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=config.cpu_cores,
)

Loading dataset: 100%|██████████| 130/130 [01:10<00:00,  1.84it/s]


In [44]:
# model = monai.networks.nets.SwinUNETR(
#     in_channels=1,
#     out_channels=2,
#     img_size=(224, 224, 96),
#     spatial_dims=3,
#     use_checkpoint=False,
#     use_v2=True,
# ).to(config.device)

In [45]:
# model = monai.networks.nets.SegResNet(
#     blocks_down=[2, 4, 4, 4],
#     blocks_up=[2, 4, 4],
#     init_filters=16,
#     in_channels=1,
#     out_channels=2,
#     dropout_prob=0.2,
# ).to(config.device)

In [3]:
model = monai.networks.nets.UNETR(
    in_channels=1,      
    out_channels=2,     
    img_size=(224, 224, 96),  
    feature_size=16,  
    hidden_size=768,  
    num_heads=16,  
    proj_type="conv",
    norm_name="instance",
).to(config.device)

In [47]:
# model = monai.networks.nets.UNet(
#     spatial_dims=3,
#     in_channels=1,
#     out_channels=2,
#     channels=(32, 64, 128, 256),
#     strides=(2, 2, 2),
#     num_res_units=2,
# ).to(config.device)

In [4]:
optimizer = torch.optim.Adam(model.parameters(), config.lr_init, weight_decay=config.weight_decay)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=config.epochs, eta_min=config.lr_min
)
loss_function = monai.losses.DiceCELoss(
    smooth_nr=config.smooth_nr,
    smooth_dr=config.smooth_dr,
    squared_pred=True,
    to_onehot_y=False,  # labels are already separated by channel
    sigmoid=True,  # 0 is background, 1 is label
    weight=torch.tensor([1.1698134, 0.8732383]).to(config.device)
)
experiment_name = "debug-UNETR"
writer = SummaryWriter(f"logs/writer/{experiment_name}")
trainer = MedSegTrainer(
    experiment_name=experiment_name,
    model=model,
    epochs=config.epochs,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    loss_f=loss_function,
    writer=writer,
    save_every=50
)

In [5]:
trainer.load_checkpoint("logs/debug-UNETR/checkpoints/4950")

/cluster/work/andrebw/medical-image-segmentation/trainer.py:209: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(f"{checkpoint_path}/mode

In [50]:
trainer.fit(train_loader=train_loader)

Epoch 1/400


Training step: 100%|██████████| 33/33 [00:40<00:00,  1.23s/it]


Loss epoch: 33.573
Epoch 2/400


Training step: 100%|██████████| 33/33 [00:35<00:00,  1.08s/it]


Loss epoch: 33.473
Epoch 3/400


...

Loss epoch: 18.240
Epoch 150/400


Training step: 100%|██████████| 33/33 [00:35<00:00,  1.07s/it]


Loss epoch: 18.464
Epoch 151/400


Training step:  12%|█▏        | 4/33 [00:08<01:01,  2.13s/it]


KeyboardInterrupt: 

In [ ]:
# model.load_state_dict(torch.load("logs/exp1-UNETR/model.pth"))

In [6]:
torch.clear_autocast_cache()
test_dataset = HNTSDataset("data/test", transform=NormalizeIntensityd(keys="image"))
test_loader = monai.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=config.cpu_cores,
)

Loading dataset:   0%|          | 0/20 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


In [13]:
metrics_dict = {
    "IoU": monai.metrics.MeanIoU(include_background=True, reduction="mean"),
    "Dice": monai.metrics.DiceMetric(include_background=True, reduction="mean"),
    "DiceBatched": monai.metrics.DiceMetric(
        include_background=True, reduction="mean_batch"
    ),
}

score = trainer.test(
    test_loader=test_loader,
    metrics=metrics_dict,
)

print(f"Metrics: {score}")

TypeError: test() missing 1 required positional argument: 'self'

In [ ]:
from monai.inferers import sliding_window_inference


def inference(model, input_):
    def _compute(input_):
        return sliding_window_inference(
            inputs=input_,
            roi_size=(224, 224, 96),
            sw_batch_size=1,
            predictor=model,
            overlap=0.2,
        )

    # with torch.amp.autocast('cuda'):
    post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    return post_trans(_compute(input_.to(config.device)))


In [11]:
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

sample_idx = 14
sample = test_dataset[sample_idx]
image = sample['image']
mask_pred = inference(model, image.unsqueeze(0).to(config.device))[0].cpu()

image = monai.transforms.Orientation(axcodes="SPL")(sample['image'])
mask_target = monai.transforms.Orientation(axcodes="SPL")(sample['mask'])
mask_pred = monai.transforms.Orientation(axcodes="SPL")(mask_pred)

def plot_slice(slice_idx):
    image_slice =  image[0,slice_idx]
    label0 = [mask_pred[0,slice_idx], mask_target[0,slice_idx]]
    label1 = [mask_pred[1,slice_idx], mask_target[1,slice_idx]]
    title = ['Prediction', 'Target']
    
    fig, axs = plt.subplots(1,2,figsize=(16,8))
    for i in range(2):
        axs[i].imshow(image_slice, cmap="gray", alpha=1.0)
        axs[i].imshow(label0[i], cmap="Reds", alpha=0.3)
        axs[i].imshow(label1[i], cmap="plasma", alpha=0.3)
        axs[i].set_title(title[i])
        axs[i].axis('off')
    plt.tight_layout()
    plt.show()
slider = widgets.IntSlider(value=mask_pred.shape[1]//2, min=0, max=mask_pred.shape[1]-1, step=1, description='Slice Index')
interact(plot_slice, slice_idx=slider);

interactive(children=(IntSlider(value=74, description='Slice Index', max=148), Output()), _dom_classes=('widge…

In [ ]:
image = sample["image"]
mask_target = sample["mask"]
mask_pred = inference(trainer.model, image.unsqueeze(0))[0].cpu()

image = monai.transforms.Orientation(axcodes="SPL")(image)
mask_target = monai.transforms.Orientation(axcodes="SPL")(mask_target)
mask_pred = monai.transforms.Orientation(axcodes="SPL")(mask_pred)

title = ["Prediction", "Target"]
for slice_idx in range(mask_target.shape[1]):
    image_slice = image[0, slice_idx]
    label0 = [mask_pred[0, slice_idx], mask_target[0, slice_idx]]
    label1 = [mask_pred[1, slice_idx], mask_target[1, slice_idx]]
    fig, axs = plt.subplots(1, 2, figsize=(16, 8))
    for i in range(2):
        axs[i].imshow(image_slice, cmap="gray", alpha=1.0)
        axs[i].imshow(label0[i], cmap="Reds", alpha=0.3)
        axs[i].imshow(label1[i], cmap="plasma", alpha=0.3)
        axs[i].set_title(title[i])
        axs[i].axis("off")
    plt.tight_layout()
    writer.add_figure("prediction_vs_target", fig, global_step=slice_idx)